In [2]:
%matplotlib inline
import matplotlib
import seaborn as sns
import pandas as pd
matplotlib.rcParams['savefig.dpi'] = 144
import numpy as np
import sys
sns.set_style("whitegrid",{'axes.grid' : False})#,{axes.grid:False})#'grid.axes',False)

In [12]:
from skimage.segmentation import slic, felzenszwalb, mark_boundaries
from skimage.util import img_as_float, img_as_ubyte
from skimage.measure import regionprops
from skimage.feature import blob_doh
from skimage import io, draw, color
import matplotlib.pyplot as plt
import cv2
import numpy as np
import argparse
import fisher2 as fv
import matplotlib.patches as mpatches

from skimage import transform as tf
from skimage.feature import CENSURE
from skimage.color import rgb2gray

from sklearn import tree, ensemble

In [4]:
main_dir = "/home/vagrant/datacourse/capstone/food-101/"
meta_dir = main_dir + 'meta/'
images_dirs = main_dir + 'images/'

classes_file = 'classes.txt'
classes_vegan_file = 'classes_vegan.txt'
train_setlist = 'train.txt'
test_setlist = 'test.txt'



In [ ]:

def loadim(file): 
    print ('reading')
    arr = img_as_float(io.imread(file))
    return arr

def segme(img):
    s=felzenszwalb(img, scale=132000, sigma=.1, min_size=2621)
    return s

def segmecol(segged,img): 
#    print ('overlaying')
    labels = segged
    labels = labels + 1  # So that no labelled region is 0 and ignored by regionprops
    regions = regionprops(labels)    
    label_rgb = color.label2rgb(labels, img, kind='overlay')
    return label_rgb


def make_mask(img,segs,pixid):
    for props in regionprops(segs, rgb2gray(img)):
        if props.label == pixid:
            fig, bax = plt.subplots(nrows=2,ncols=1,figsize=(24,24))
            #mask_seg_ind = cv2.cvtcolor(im, cv2.COLOR_BGR2GRAY)
            mask_seg_ind = np.zeros(im[:,:,0].shape)
            minr, minc, maxr, maxc = props.bbox
            rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                                  fill=False, edgecolor='red', linewidth=2)

            mask_seg_ind[minr:maxr, minc:maxc] = 1
            mask_seg_ind = mask_seg_ind.astype(np.uint8)
            np.uint8(mask_seg_ind)
            mask_seg_ind = cv2.convertScaleAbs(mask_seg_ind)
    return mask_seg_ind

def get_features(img, mask):
#    mask = greyed_bbox_img * 0 + 1
    gray_im = cv2.cvtColor(img_as_ubyte(img)*mask,cv2.COLOR_RGB2BGR)
    gray_im = cv2.cvtColor(gray_im,cv2.COLOR_BGR2GRAY)
    dense_surf = cv2.FeatureDetector_create("Dense")
    kp = dense_surf.detect(gray_im, mask_seg_ind)
    desc_ext = cv2.DescriptorExtractor_create("Dense")
    des = desc_ext.compute(gray_im*mask_seg_ind, kp)
    return kp, des

def trasnform_feats(desc):
    desc_tran = [ [np.sqrt(y) if y>0 else -np.sqrt(-1*y) for y in x]/sum(x) for x in desc ]
    return desc_tran

def get_lab_feats(img, mask):
    newimg = cv2.cvtColor(img*mask, cv2.COLOR_RGB2BGR)
    newimg = cv2.cvtColor(newimg, cv2.COLOR_BGR2LAB)
    lab_values = newimg.reshape(len(newimg),1)
    return lab_values

def make_fishvec(kp,des_tran,lab_feats_tran):
    args = fv.get_args()
    working_folder = args.dir

    gmm = fv.generate_gmm(kp,des_tran,lab_feats_tran)
    fisher_features = fv.fisher_features(gmm)
    classifier = train(gmm, fisher_features)
    rate = fv.success_rate(classifier, fisher_features)
    return fisher_features

    
im = loadim() 
#im = im[:,:,0]
#print len(im), type(im), im.shape

segs = segme(im)
labels = segs + 1
for props in regionprops(segs, rgb2gray(im)):
    #if props.area == 61927:9667
    if props.area == 61927:
        print "props label,area are: ", props.label, props.area
        fig, bax = plt.subplots(nrows=2,ncols=1,figsize=(24,24))
        #mask_seg_ind = cv2.cvtcolor(im, cv2.COLOR_BGR2GRAY)
        mask_seg_ind = np.zeros(im[:,:,0].shape)
        minr, minc, maxr, maxc = props.bbox
        rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                              fill=False, edgecolor='red', linewidth=2)
        
        
        mask_seg_ind[minr:maxr, minc:maxc] = 1
        mask_seg_ind = mask_seg_ind.astype(np.uint8)
        np.uint8(mask_seg_ind)
        mask_seg_ind = cv2.convertScaleAbs(mask_seg_ind)
        print mask_seg_ind.dtype
        print type(mask_seg_ind)

        #gray_im = rgb2gray(im)*255
        gray_im = cv2.cvtColor(img_as_ubyte(im),cv2.COLOR_RGB2BGR)
        gray_im = cv2.cvtColor(gray_im,cv2.COLOR_BGR2GRAY)
        print gray_im[:2]
#        gray_im = gray_im.astype(int)    
        print mask_seg_ind.shape, gray_im.shape
        surf = cv2.SURF(1500,extended=True, upright=False)
        print surf.hessianThreshold
        kp, des = surf.detectAndCompute(gray_im, mask_seg_ind)
        des_rf = [ [np.sqrt(y) if y>0 else -np.sqrt(-1*y) for y in x]/sum(x) for x in des ]
        print "descriptors info: ", len(des), type(des), des.shape
        print "kp attributes: ", kp[0].pt, kp[0].size, kp[0].angle
        print surf.descriptorSize()
        print "uprigt? ",surf.upright
        dense_surf = cv2.FeatureDetector_create("Dense")
#        kp = dense_surf.detect(gray_im, mask_seg_ind)
#        desc_ext = cv2.DescriptorExtractor_create("Dense")
#        des = desc_ext.compute(gray_im*mask_seg_ind, kp)
        cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS = True
        im2 = cv2.drawKeypoints(img_as_ubyte(im),kp,color=(0,0,255),flags=4)#cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        print "flag: ", cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
        
        bax[0].add_patch(rect)
#        bax[0].imshow(cv2.drawKeypoints(gray_im*mask_seg_ind,kp,4)); #plt.show()
        bax[0].imshow(cv2.drawKeypoints(gray_im,kp,color=(255,0,0),flags=4))#cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)); #plt.show()
        bax[1].imshow(im2); plt.show()
#        print mask_seg_ind[minr:maxr,minc:maxc]
#        kps, des = get_features(gray_im, mask_seg_ind)

        
#        props = regionprops(segs[np.where(segs==segid)], rgb2gray(im[np.where(segs==segid)]))
#        print "label of region with size 108446 is: ", props.label        
#        kps, des = get_features(props.image)
        break
        
image_label_overlay = segmecol(segs,im)
fig, ax=plt.subplots(nrows=1, ncols=1, figsize=(8,8))
ax.imshow(image_label_overlay)
plt.show()


In [ ]:
# train and test below
def process_data(train_test):
    feats_vec = []
    if train_test == 'train':
        loi = open(main_dir + meta_dir + train_setlist,'r')
        compute_labels = True
        labs = []
        vdict = dict()
        counter = 0
        for line in open(main_dir + meta_dir + classes_vegan_file,'r'):
            key,val = line.strip().split()
            vdict[key] = (counter,val)
            counter += 1
        vlabs = []
        feats_vec = []
    else: 
        loi = open(main_dir + meta_dir + test_setlist,'r')
        compute_labels = False
        
    filelist = loi.read().split("\n")
    
    for f in filelist: 
        full_fname = main_dir+images_dir+f+'.jpg'
        img = load_im(full_fname)
        supixes = segme(img)
        
        for pixid in np.unique(suppixes):
            mask = make_mask(pixid)
            kp,des = get_features(gr_im, mask)
            des_tran = trasnform_feats(des)
            lab_feats = get_lab_feats(im, mask)
            lab_feats_tran = transform_lab(lab_feats)
            fishvec = make_fishvec(kp,des_tran,lab_feats_tran)
            feats_vec += fishvec
            if compute_labels:
                cname = f.split("\/")[0]
                labs += vdict[cname][0]
                vlabs += vdict[cname][1]
               
    # component mining
    inds = np.random.randint(0,2400000,2400000)
    validation_inds = inds[0:200000]
    validation_set = feats_vec[validation_inds]
    if compute_labels:
        validation_labels = labs[validation_inds]
        validation_vlables = vlabs[validation_inds]
    for treeid in range(1,2400000/200000):
        tree_inds = inds[treeid*200000, treeid*200000+200000]
        tree_feats = feats_vec[treeinds]
        if compute_labels:
            tree_labs = labs[tree_inds]
            tree_vlabs = vlabs[tree_inds]
        
    rf = ensemble.RandomForestClassifier(n_estimators=12,criterion='entropy',min_samples_split=25,max_depth=7,random_state=53,max_features=30)
    order = [i[0] for i in sorted(enumerate(rf.feature_importances_), key=lambda x:-x[1])][:20]
    newfeats = [arr[order] for arr in feats_vec]                                         
    if compute_labels:                                         
        return newfeats, labs, vlabs
    else:
        return newfeats                                           

In [5]:
# train 
feats, labels, vlabels = process_data(train_test='train')
classifier = ensemble.RandomForestClassifier(n_estimators=30,criterion='entropy',min_samples_split=25,max_depth=7,random_state=53,max_features=20
pred_labs = classifier.fit(feats,labels)
pred_vlabs = rf.fit(feats,vlabels)
success_rate_labs = fv.success_rate(feats,pred_labels,process_data(train_test='test'))                                     
success_rate_vlabs = fv.success_rate(feats,pred_vlabels,process_data(train_test='test'))                                     